# Динамика Universe и Sample
Пример расчета численности целевых аудиторий в населении и выборки в динамике

## Описание задачи и условий расчета
- Период: 01.06.2022 - 31.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+, Все 12-24, Все 25-34, Все 35-44, Все 45+, Все 20-44, Все 20+, Мужчины, Женщины, Мужчины 20+, Женщины 20+, Мужчины 20-44, Женщины 20-44, Мужчины 35+, Женщины 35+
- Место просмотра: Все места (дом+дача)							
- Статистики: Universe, Sample

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике демографических атрибутов найдем необходимые нам переменные
cats.get_tv_demo_attribute(names=['пол','возраст'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: sex, age

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-06-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы: выбор канала не влияет на расчет Universe и Sample
company_filter = None

# Указываем список статистик для расчета
statistics = ['Universe000', 'Sample']

# Указываем срезы: месяц
slices = ['researchMonth']

# Задаем условия сортировки
sortings = None

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Сформируем словарь целевых аудиторий

In [ ]:
# Задаем необходимые группы
targets = {
    'a. Все 4+':'age >= 4',
    'b. 4-24':'age >= 4 AND age <= 24',
    'c. 25-34':'age >= 25 AND age <= 34',
    'd. 35-44':'age >= 35 AND age <= 44',
    'e. 45+':'age >= 45',
    'f. 20-44':'age >= 20 AND age <= 44',
    'g. 20+':'age >= 20',
    'h. Мужчины ':'sex = 1',
    'i. Женщины':'sex = 2',
    'j. Мужчины 20+':'sex = 1 AND age >=20',
    'k. Женщины 20+':'sex = 2 AND age >=20',
    'l. Мужчины 20-44':'sex = 1 AND age >=20 AND age <= 44',
    'm. Женщины 20-44':'sex = 2 AND age >=20 AND age <= 44',
    'n. Мужчины 35+':'sex = 1 AND age >=35',
    'o. Женщины 35+':'sex = 2 AND age >=35'    
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры
    project_name = target
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name']+slices+statistics]

df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# округляем семпл до целого
df['Sample'] = df['Sample'].round(0)
df = df.astype({'Sample': 'int'})

In [ ]:
# Формируем сводную таблицу: строки - ЦА, столбцы - периоды, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['prj_name'], 
                        columns=['researchMonth'])
df

In [ ]:
# Опционально: поменять местами уровни в столбцах
df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('04_universe&sample_by_months')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)